In [1]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf

2023-05-24 11:03:37.114738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dev = qml.device('default.mixed', wires=3)

In [3]:
def real(angles,**kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles,wires=0)

In [10]:
def generator(w,**kwargs):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)
    qml.BitFlip(p, wires=0)
    qml.BitFlip(p, wires=1)

In [11]:
def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)
    qml.BitFlip(p, wires=0)
    qml.BitFlip(p, wires=2)

In [12]:
@qml.qnode(dev, interface='tf')
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi,theta,omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

@qml.qnode(dev,interface='tf')
def gen_disc_circuit(gen_weights,disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

In [13]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi,theta,omega,disc_weights)
    #convert to probability
    prob_real_true = (true_disc_output+1)/2
    return prob_real_true

In [14]:
def prob_fake_true(gen_weights,disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights,disc_weights)
    #convert to probability
    prob_fake_true = (fake_disc_output+1)/2
    return prob_fake_true

In [15]:
def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights,disc_weights)-prob_real_true(disc_weights)
    return cost
def gen_cost(gen_weights):
    cost = -prob_fake_true(gen_weights,disc_weights)
    return cost

In [17]:
p = 0.1
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))

init_disc_weights = np.random.normal(size=(9,))
gen_weights = tf.Variable(init_gen_weights,dtype=tf.float64)
disc_weights = tf.Variable(init_disc_weights,dtype=tf.float64)

In [18]:
opt = tf.keras.optimizers.experimental.SGD(learning_rate=0.4)

var = disc_weights
print('var =',var)

loss = lambda: disc_cost(disc_weights)         # d(loss)/d(var1) = var1
print('loss =',disc_cost(disc_weights) )

for step in range(50):
    opt.minimize(loss, [var])
    if step % 5 == 0:
        loss_val = loss().numpy()
        print("Step {}: cost = {}".format(step, loss_val))

var = <tf.Variable 'Variable:0' shape=(9,) dtype=float64, numpy=
array([ 0.4105985 ,  0.14404357,  1.45427351,  0.76103773,  0.12167502,
        0.44386323,  0.33367433,  1.49407907, -0.20515826])>
loss = tf.Tensor(-0.018053712648460063, shape=(), dtype=float64)
Step 0: cost = -0.04039486732759001
Step 5: cost = -0.16955382257436336
Step 10: cost = -0.30050000693654755
Step 15: cost = -0.3624121623436303
Step 20: cost = -0.38101630095055294
Step 25: cost = -0.38778099174893677
Step 30: cost = -0.391475234185672
Step 35: cost = -0.39394727548181707
Step 40: cost = -0.39567028538167304
Step 45: cost = -0.39686166690611147


In [19]:
opt = tf.keras.optimizers.legacy.SGD(learning_rate=0.4)
var_g = gen_weights
print('var_g =',var_g)

loss = lambda: gen_cost(gen_weights)         # d(loss)/d(var1) = var1
print('loss =',gen_cost(gen_weights))

for step in range(50):
    opt.minimize(loss, [var_g])
    if step % 5 == 0:
        loss_val = loss().numpy()
        print("Step {}: cost = {}".format(step, loss_val))

var_g = <tf.Variable 'Variable:0' shape=(9,) dtype=float64, numpy=
array([ 3.15923318e+00,  4.00157208e-03,  9.78737984e-03,  2.24089320e-02,
        1.86755799e-02, -9.77277880e-03,  9.50088418e-03, -1.51357208e-03,
       -1.03218852e-03])>
loss = tf.Tensor(-0.4999885750980042, shape=(), dtype=float64)
Step 0: cost = -0.5369006988043046
Step 5: cost = -0.6961265890726314
Step 10: cost = -0.7786430424654182
Step 15: cost = -0.8081364125621053
Step 20: cost = -0.8189018262844825
Step 25: cost = -0.8235605265277395
Step 30: cost = -0.8258495109301165
Step 35: cost = -0.8270537190103742
Step 40: cost = -0.8277100647121625
Step 45: cost = -0.8280747525256973


In [20]:
print("Prob(fake classified as real): ", prob_fake_true(var_g, var).numpy())

Prob(fake classified as real):  0.8282474608727748


In [21]:
print("Discriminator cost: ", disc_cost(var).numpy())

Discriminator cost:  -0.06927626049472946


In [22]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

@qml.qnode(dev, interface="tf")
def bloch_vector_real(angles):
    real(angles)
    return [qml.expval(o) for o in obs]

@qml.qnode(dev, interface="tf")
def bloch_vector_generator(angles):
    generator(angles)
    return [qml.expval(o) for o in obs]

print(f"Real Bloch vector: {bloch_vector_real([phi, theta, omega])}")
print(f"Generator Bloch vector: {bloch_vector_generator(var_g)}")

Real Bloch vector: [tensor(-0.21694187, requires_grad=True), tensor(0.45048443, requires_grad=True), tensor(-0.8660254, requires_grad=True)]
Generator Bloch vector: [<tf.Tensor: shape=(), dtype=float64, numpy=-0.3596466819965153>, <tf.Tensor: shape=(), dtype=float64, numpy=0.30033131263670876>, <tf.Tensor: shape=(), dtype=float64, numpy=-0.6827290020322043>]


In [24]:
qml.math.fidelity(bloch_vector_real([phi, theta, omega]), bloch_vector_generator(var_g))

0.6473452183799383